In [1]:
import torch
import pandas as pd
from dataset import get_test_dataset
from efficientnet_pytorch import EfficientNet
from inference import create_label
from torch.utils.data import DataLoader

In [2]:
df_test_path = '/opt/ml/input/data/eval/info.csv'
df_test = pd.read_csv(df_test_path)
test_dataset = get_test_dataset(df_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3, drop_last=False)

In [3]:
def sum_label(df):
    
    def _label(row):
        return row['age'] + 3*row['label']# + 6*row['mask']

    df['ans'] = df.apply(_label, axis=1)
    return df[['ImageID', 'ans']]

In [4]:
device = 'cuda'

In [5]:
## add mask_and gender label

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=6).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/split_test/best_30_log7.pt'))
df_submit = create_label(model, test_dataloader, df_test.copy(), device, target='label')

Loaded pretrained weights for efficientnet-b0


100%|██████████| 197/197 [00:24<00:00,  7.97it/s]

inference label complete!


In [6]:
## add age label

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/age/extended_far_best.pt'))
df_submit = create_label(model, test_dataloader, df_submit, device, target='age')

Loaded pretrained weights for efficientnet-b0


100%|██████████| 197/197 [00:24<00:00,  7.99it/s]

inference age complete!


In [ ]:
# model.load_state_dict(torch.load('/opt/ml/code/save/mask/6e5_9987.pt'))
# df_submit = create_label(model, test_dataloader, df_submit, device, target='mask')

# model.fc = nn.Linear(model.fc.in_features, 2).cuda()
# model.load_state_dict(torch.load('/opt/ml/code/save/gender/6e5_9851.pt'))
# df_submit = create_label(model, test_dataloader, df_submit, device, target='gender')

In [7]:
## sumup labels

df_submit = sum_label(df_submit)

In [8]:
df_submit.to_csv('submit_30.csv', index=False)